## Migration from Text Completions API to the new Messages API

The text completions API for Claude is now in legacy mode and it is recommended to migrate to 
the new Messages API. The Messages API is backward compatible with current Claude Models on Bedrock and requires that a modified JSON body be passed to the invoke methods. An example of a text completion API style body and prompt is shown below:

prompt='Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n'

body = json.dumps({
                "prompt": prompt,
                "temperature": 0.5,
                "top_p": 0.9,
                "max_tokens_to_sample": 512,
            })

The new body will have the following form with an updated messages element.

body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "temperature": 0.5,
            "top_p": 0.9,
            "system": system_prompt,
            "messages": messages
        }  
    ) 
    
The messages element represents a list of JSONL objects that include the role and content for each turn in an input. For a single input, the messages element is shown below where the "Human" tag from the text Completion API is replaced by "user" in the new API:

messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

Lets see how this works with some examples with Claude 2.1

### Initialize packages

In [ ]:
%pip install --upgrade pip
%pip install boto3>=1.33.2 --force-reinstall --quiet
%pip install botocore>=1.33.2 --force-reinstall --quiet


#### Restart the kernel with the updated packages that are installed through the dependencies above

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [36]:
import boto3
import json
import base64

bedrock_client = boto3.client('bedrock-runtime',region_name='us-west-2')


### Helper function to pass our models, messages, and inference parameters

In [37]:
def generate_message(bedrock_runtime, model_id, messages, max_tokens,top_p,temp):

    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

### Single-Input

In [ ]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

### Multi-turn Input

In [ ]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = 'anthropic.claude-v2:1',messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

### Claude 3 with Text Completions API will give the following error: 

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: "claude-3-sonnet-20240229" is not supported on this API. Please use the Messages API instead.

In [38]:
prompt='Human:\n\n What is quantum mechanics? \n\n Assistant:\n\n'

body = json.dumps({ "prompt": prompt, "temperature": 0.5, "top_p": 0.9, "max_tokens_to_sample": 512, })

response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")

ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: "claude-3-sonnet-20240229" is not supported on this API. Please use the Messages API instead.

### Claude 3-sonnet single-turn example with correct Messages API call

In [ ]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics?"}]}]

generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9)

### Claude 3-sonnet multi-turn example with correct Messages API call

In [ ]:
messages=[{ "role":'user', "content":[{'type':'text','text': "What is quantum mechanics? "}]},\
         { "role":'assistant', "content":[{'type':'text','text': "It is a branch of physics that \
         describes how matter and energy interact with discrete energy values "}]},\
         { "role":'user', "content":[{'type':'text','text': "Can you explain a bit more about discrete energies?"}]}]

generate_message(bedrock_client, model_id = "anthropic.claude-3-sonnet-20240229-v1:0",messages=messages,max_tokens=512,temp=0.5,top_p=0.9)